## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-05-06-54-29 +0000,bbc,US attorney general orders grand jury hearings...,https://www.bbc.com/news/articles/c206dv2ekdeo...
1,2025-08-05-06-49-40 +0000,nypost,Denmark zoo asks people to donate their pets a...,https://nypost.com/2025/08/05/world-news/denma...
2,2025-08-05-06-33-00 +0000,wsj,Australia Selects Japan’s Mitsubishi Heavy to ...,https://www.wsj.com/business/australia-selects...
3,2025-08-05-06-00-33 +0000,wapo,"Five years after huge Beirut blast, justice st...",https://www.washingtonpost.com/world/2025/08/0...
4,2025-08-05-05-49-29 +0000,latimes,"Rosa fire burns 1,200 acres in Riverside Count...",https://www.latimes.com/california/story/2025-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2353/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
5,trump,51
87,new,21
169,gaza,12
306,house,12
576,will,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
86,2025-08-04-22-54-00 +0000,wsj,Democratic lawmakers blocked the Texas House f...,https://www.wsj.com/politics/policy/republican...,125
35,2025-08-05-01-35-00 +0000,wsj,Trump’s firing of the top Bureau of Labor Stat...,https://www.wsj.com/finance/investing/trump-bl...,109
72,2025-08-04-23-23-00 +0000,wsj,President Trump’s decision to fire the head of...,https://www.wsj.com/economy/real-strains-insid...,104
48,2025-08-05-01-00-00 +0000,wsj,President Trump’s high-stakes push to create m...,https://www.wsj.com/politics/texas-redistricti...,95
198,2025-08-04-16-50-00 +0000,wsj,What has Trump done as president so far? The W...,https://www.wsj.com/politics/policy/donald-tru...,95


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
86,125,2025-08-04-22-54-00 +0000,wsj,Democratic lawmakers blocked the Texas House f...,https://www.wsj.com/politics/policy/republican...
35,58,2025-08-05-01-35-00 +0000,wsj,Trump’s firing of the top Bureau of Labor Stat...,https://www.wsj.com/finance/investing/trump-bl...
112,57,2025-08-04-21-48-14 +0000,nypost,Brazil’s Supreme Court orders house arrest for...,https://nypost.com/2025/08/04/world-news/brazi...
220,49,2025-08-04-15-16-00 +0000,wsj,Elon Musk Gets $23.7 Billion Stock Award From ...,https://www.wsj.com/business/autos/tesla-appro...
286,47,2025-08-04-08-00-20 +0000,cbc,The nuclear arms race is heating up. Will it t...,https://www.cbc.ca/radio/sunday/hiroshima-naga...
167,43,2025-08-04-18-44-19 +0000,nypost,Netanyahu approves plan to fully occupy Gaza S...,https://nypost.com/2025/08/04/world-news/netan...
46,42,2025-08-05-01-03-45 +0000,nypost,Clip of Sydney Sweeney’s expert shooting range...,https://nypost.com/2025/08/04/us-news/clip-of-...
40,41,2025-08-05-01-17-08 +0000,latimes,Wealthy first-time candidate Cloobeck drops $1...,https://www.latimes.com/california/story/2025-...
51,38,2025-08-05-00-50-22 +0000,nypost,Oil baron CEO claims he was subject of ‘sham’ ...,https://nypost.com/2025/08/04/us-news/oil-baro...
164,37,2025-08-04-19-06-46 +0000,nypost,Far left Sen. Warren gushes over NYC mayoral c...,https://nypost.com/2025/08/04/us-news/far-left...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
